# Extract the DrumMidis.zip and SavedSession**.zip files

In [1]:
print(1)

1


In [2]:
import zipfile
import os
import bz2
from umap import UMAP

from bokeh.palettes import inferno, Category20b
from bokeh.core.enums import MarkerType
from bokeh.plotting import figure, show, save
from bokeh.io import output_notebook, reset_output
# output_notebook()


def extract_zip(zip_file, extract_dir):
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)

def extract_bz2(bz2_file, extract_dir):
    """
    Extract a BZ2 compressed file.
    
    :param bz2_file: Path to the BZ2 compressed file.
    :param extract_dir: Directory where the decompressed contents will be saved.
    """
    # Ensure the extraction directory exists
    os.makedirs(extract_dir, exist_ok=True)

    # Open the BZ2 file for reading
    with bz2.open(bz2_file, 'rb') as file_in:
        # Decompress the data
        data = file_in.read()
        
        # Write the decompressed data to the output file
        with open("temp.zip", 'wb') as file_out:
            file_out.write(data)
        
        extract_zip("temp.zip", extract_dir)
        
        os.remove("temp.zip")
    
if not os.path.exists('DrumMidis'):
    extract_zip('DrumMidis.zip', 'DrumMidis')
    
if not os.path.exists('SavedSessions'):
    extract_bz2('SavedSessions.bz2', 'SavedSessions')

# Load Bongosero Collection


```
collection 
|
|------------ User1
|            |------------ Attempts
|            |            |------------ Attempt1
|            |            |------------ Attempt2
|            |            |------------ ...
```


In [3]:
from API import ElBongoseroCollection

collection = ElBongoseroCollection('SavedSessions/SavedSessions')
collection


/Users/behzadhaki/Github/GrooveTransformer/GrooveTransformerVenv/lib/python3.9/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


ElBongoseroCollection with 3286 users, total of 6640 attempts

# Subsetting

You can Subset using the following methods, the returned subset will also be of type `ElBongoseroCollection`

Also, subsetters can be cascaded one after the other

Available subsetters:
- `filter_by_assessment_duration_minimumm(min_duration)` --> the amount user thought about rating their performance
- `filter_by_attempt_duration_minimum(min_duration)` --> the amount of time the user spent on the attempt
- `filter_by_self_assessment_within_range(min_rating, max_rating)` --> the user's self assessment of their performance
- `filter_by_tempo_range(min_tempo, max_tempo)` --> the tempo of the drum loop
-  `filter_by_total_bongo_hits_within_range(min_hits, max_hits)` --> the total number of bongo hits in the loop (use to get rid of empty loops)
- `filter_by_style(style)` --> the style of the drum loop
- `filter_by_user_level_of_musical_experience(min_, max_)` --> the user's level of musical experience
- `filter_by_user_exhibion_rating(min_, max_)` --> the user's rating of their experience with the installation
- 

In [4]:
collection[0][0]

{'attempt_duration': 9.0, 'self_assessment': 1, 'assessment_time': 12.0, 'attempt_tempo': 122.0, 'drum_path': 'DrumMidis/Afrocuban/p0000003354__m001__beat__afrocuban__066---068.mid', 'metadata_json': 'SavedSessions/SavedSessions/session_00003119--2023-11-29--14-11-34/Part2_BongosAlonWithDrums/attempt_001/groove_metadata.json', 'genre': 'Afrocuban', 'user_level_of_musical_experience': 5, 'user_exhibion_rating': 5}

In [5]:
# available styles
styles = collection.get_all_styles()

subset = collection.filter_by_style('Rock').filter_by_user_level_of_musical_experience(1, 1)

subset

ElBongoseroCollection with 106 users, total of 111 attempts

In [ ]:
# Create source drums mapping

source_drums = []
for user_data in collection:
    for attempt_data in user_data.attempts:
        source_drums.append(attempt_data.drum_path)
        attempt_data
source_drums = sorted(list(set(source_drums)))

source_drums_per_style = {}
for i, drum in enumerate(source_drums):
    style = drum.split('/')[1]
    if style not in source_drums_per_style:
        source_drums_per_style[style] = {}
    n_drums_so_far = len(list(source_drums_per_style[style].keys()))
    source_drums_per_style[style][drum] = f'source_{n_drums_so_far}'

# source_drums_mapping = {}
# for i, drum in enumerate(source_drums):
#     style = drum.split('/')[1]
#     source_drums_mapping[drum] = (f'source_{i}', style)
    
source_drums_per_style['Afrocuban'][collection[0][0].drum_path]
source_drums_per_style

In [ ]:
# # put all source drums
# for style in styles:
#     subset_style = collection.filter_by_style(style)
#     for experience in range(1, 6):
#         subset = subset_style.filter_by_user_level_of_musical_experience(experience, experience)
#         cnt = 0
#         for i, user_data in enumerate(subset):
#             for j, attempt_data in enumerate(user_data.attempts):
#                 try:
#                     cnt += 1
#                     # hvo_seq_drum = attempt_data.load_source_drum_hvo_seq()
#                     # hvo_seq_drum.save_hvo_to_midi(f"midi_organized_by_experience/{style}/experience_{experience-1}/{cnt}/drum.mid")
#                     hvo_seq_bongo = attempt_data.load_bongo_loop_hvo_seq()
#                     hvo_seq_bongo.save_hvo_to_midi(f"midi_organized_by_experience/{style}/experience_{experience-1}/{cnt}/bongo.mid")
#                     hvo_seq_drum_and_bongo = attempt_data.load_drums_with_bongos_hvo_sequence()
#                     hvo_seq_drum_and_bongo.save_hvo_to_midi(f"midi_organized_by_experience/{style}/experience_{experience-1}/{cnt}/drum_and_bongo.mid")
#                 except Exception as e:
#                     print(e)
#                     continue

In [ ]:
# for style in styles:
#     subset_style = collection.filter_by_style(style)
#     for experience in range(1, 6):
#         subset = subset_style.filter_by_user_level_of_musical_experience(experience, experience)
#         cnt = 0
#         for i, user_data in enumerate(subset):
#             for j, attempt_data in enumerate(user_data.attempts):
#                 try:
#                     cnt += 1
#                     drum_name, style = source_drums_mapping[attempt_data.drum_path]
#                     hvo_seq_drum = attempt_data.load_source_drum_hvo_seq()
#                     # hvo_seq_drum.save_hvo_to_midi(f"midi_organized_by_source_drums/{drum_name}/{style}/experience_{experience-1}/{cnt}/drum.mid")
#                     hvo_seq_drum.save_hvo_to_midi(f"midi_organized_by_source_drums/{style}/{drum_name}/drum.mid")
                    
#                 except Exception as e:
#                     print(e)
#                     continue
    

In [ ]:
import glob
import tqdm

for style in tqdm.tqdm(styles):
    subset_style = collection.filter_by_style(style)
    for experience in range(1, 6):
        subset = subset_style.filter_by_user_level_of_musical_experience(experience, experience)
        cnt = 0
        for i, user_data in enumerate(subset):
            for j, attempt_data in enumerate(user_data.attempts):
                try:
                    cnt += 1
                    drum_path = attempt_data.drum_path
                    style = drum_path.split('/')[1]
                    
                    drum_name = source_drums_per_style[style][attempt_data.drum_path]

                    hvo_seq_drum = attempt_data.load_source_drum_hvo_seq()
                    hvo_seq_drum.save_hvo_to_midi(os.path.join(f"midi_organized_by_source_drums/{style}/{drum_name}/drum.mid"))

                    path_ = f"midi_organized_by_source_drums/{style}/{drum_name}/experience_{experience-1}/"
                    attempt_cnt_so_far = len(glob.glob(os.path.join(path_, '*.mid'))) // 2
                    
                    hvo_seq_bongo = attempt_data.load_bongo_loop_hvo_seq()
                    hvo_seq_bongo.save_hvo_to_midi(os.path.join(path_, f"bongo_{attempt_cnt_so_far}.mid"))
                    hvo_seq_drum_and_bongo = attempt_data.load_drums_with_bongos_hvo_sequence()
                    hvo_seq_drum_and_bongo.save_hvo_to_midi(os.path.join(path_, f"bongo_and_drum_{attempt_cnt_so_far}.mid"))
                except Exception as e:
                    print(e)
                    continue
    